In [ ]:
import pandas as pd
import os
import sys
root_dir = os.path.dirname(os.path.dirname(os.getcwd())) ## TODO: find way to put this into some global settings
if root_dir not in sys.path:
    sys.path.append(root_dir)

from references import common_cfg, istat_kpi
from src.models.factories import SchoolFactory

In [ ]:
# prepare dati affluenza
afflStatali = pd.read_csv('../../data/raw/cultura/stat_affluenzaScuole1617.csv')
afflParitarie = pd.read_csv('../../data/raw/cultura/parit_affluenzaScuole1617.csv')
affluenza = pd.concat([afflStatali, afflParitarie], axis=0)

In [ ]:
for cittaScelta in common_cfg.cityList:
    # Join con affluenza:
    inputAnagraficaFile = '../../data/interim/%s_anagrafica_scuole_geoloc.csv' % cittaScelta
    anagraficaGeoloc = pd.read_csv(inputAnagraficaFile, sep=';', decimal=',').set_index('CODICESCUOLA')
    if any(anagraficaGeoloc.Lat.isnull()):
        print('Dropping %i records as they are not geolocalized' % sum(anagraficaGeoloc.Lat.isnull()))
        anagraficaGeoloc = anagraficaGeoloc[~anagraficaGeoloc.Lat.isnull()] # drop missing geoloc
    assert not any(anagraficaGeoloc.Lat.isnull()), 'Missing geoloc info'

    # validate join 
    bDuplicated = anagraficaGeoloc.duplicated()
    if any(bDuplicated):
        print('Dropping these records as they are duplicated:')
        print(anagraficaGeoloc[bDuplicated])
        anagraficaGeoloc = anagraficaGeoloc[~bDuplicated]      
        anagraficaGeoloc.merge(affluenza, how='inner', left_index=True, right_on='CODICESCUOLA',
                    sort=False, suffixes=('_anagr', '_affl'), copy=True, validate='1:m')
    
    # print the available fraction of student data
    bFound  = anagraficaGeoloc.index.isin(affluenza.CODICESCUOLA.unique())
    print(bFound.mean())
    
    # investigate missing values
    missingSchools = anagraficaGeoloc[~bFound]
    print(missingSchools.groupby('DESCRIZIONETIPOLOGIAGRADOISTRUZIONESCUOLA').count())
    
    # check school types
    schoolTypes = affluenza[['CODICESCUOLA', 'ORDINESCUOLA']].drop_duplicates().set_index('CODICESCUOLA')
    assert not any(schoolTypes.index.duplicated()), 'Inconsistent school types'
    
    # compute join values
    alunniSum = affluenza.groupby('CODICESCUOLA')['ALUNNI', ].sum()
    datiJoin = anagraficaGeoloc.merge(pd.concat([schoolTypes, alunniSum],axis=1), 
                                      how='inner', left_index=True, right_index=True,
                sort=False, suffixes=('_anagr', '_affl'), copy=True, validate='1:1')
    
    # export to file
    datiJoin.to_csv('../../data/processed/%s_scuole.csv'%cittaScelta, 
                sep=';', decimal=',', index_label=SchoolFactory.idCol)